<a href="https://colab.research.google.com/github/asigalov61/Incredible-REMI-Piano/blob/master/Incredible_REMI_Piano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Try REMI Pop Music Transformer yourself with this simple Google Colab

In [ ]:
%tensorflow_version 1.x
!pip install gast==0.2.2
!pip install tensorflow-gpu==1.14.0
!pip install keras
!pip install miditoolkit
!git clone https://github.com/asigalov61/remi.git

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.chdir('remi')
from model import PopMusicTransformer
from glob import glob

In [ ]:
!wget 'https://incredibleremipiano.s3-us-west-1.amazonaws.com/REMI-tempo-checkpoint.zip'
!unzip 'REMI-tempo-checkpoint.zip'

In [ ]:
model = PopMusicTransformer(checkpoint='REMI-tempo-checkpoint', is_training=False)

In [ ]:
    # generate from scratch
    model.generate(
        n_target_bar=8,
        temperature=0.8,
        topk=1,
        output_path='./result/from_scratch.midi',
        prompt=None)

In [ ]:
    # generate continuation
    model.generate(
        n_target_bar=8,
        temperature=0.8,
        topk=1,
        output_path='./result/continuation.midi',
        prompt='/content/remi/prompt1.mid')

In [ ]:
# close model
model.close()

###Finetuning

In [ ]:
#@title Alex Piano Only Drafts Original 1500 MIDIs 
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'

In [ ]:
    # declare model
    model = PopMusicTransformer(
        checkpoint='REMI-tempo-checkpoint',
        is_training=True)
    # prepare data
    midi_paths = glob('/content/remi/*.mid') # you need to revise it
    training_data = model.prepare_data(midi_paths=midi_paths)

    # check output checkpoint folder
    ####################################
    # if you use "REMI-tempo-chord-checkpoint" for the pre-trained checkpoint
    # please name your output folder as something with "chord"
    # for example: my-love-chord, cute-doggy-chord, ...
    # if use "REMI-tempo-checkpoint"
    # for example: my-love, cute-doggy, ...
    ####################################
    output_checkpoint_folder = 'REMI-finetune' # your decision
    if not os.path.exists(output_checkpoint_folder):
        os.mkdir(output_checkpoint_folder)
    
    # finetune
    model.finetune(
        training_data=training_data,
        output_checkpoint_folder=output_checkpoint_folder)

    ####################################
    # after finetuning, please choose which checkpoint you want to try
    # and change the checkpoint names you choose into "model"
    # and copy the "dictionary.pkl" into the your output_checkpoint_folder
    # ***** the same as the content format in "REMI-tempo-checkpoint" *****
    # and then, you can use "main.py" to generate your own music!
    # (do not forget to revise the checkpoint path to your own in "main.py")
    ####################################

    # close
    model.close()